In [1]:
!rm -rf ./trboard/

In [2]:
import os
import tensorflow as tf
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/talipov/MyProjects/projectX/acosta_env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
with tf.name_scope('number_embedding'):
    embedding = tf.Variable(
        np.random.uniform(size=(10,5)).astype(np.float32),
        name='embedding'
    )

In [4]:
embedding.name

'number_embedding/embedding:0'

In [5]:
with tf.name_scope('x'):
    x1 = tf.placeholder(shape=[None], dtype=tf.int32)
    x2 = tf.placeholder(shape=[None], dtype=tf.int32)
    
    x1_embedded = tf.nn.embedding_lookup(embedding, x1)
    x2_embedded = tf.nn.embedding_lookup(embedding, x2)
    
    x = tf.concat([x1_embedded, x2_embedded], axis=1)
    
with tf.name_scope('y'):
    y = tf.placeholder(shape=[None, 1], dtype=tf.float32)

In [6]:
with tf.name_scope('hidden_layer'):
    out_layer = tf.layers.Dense(
        activation=None, 
        units=1,
    )

In [7]:
with tf.name_scope('y_pred'):
    y_pred = out_layer(x)

In [8]:
y_pred, y

(<tf.Tensor 'y_pred/dense/BiasAdd:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'y/Placeholder:0' shape=(?, 1) dtype=float32>)

In [18]:
out_layer.weights

[<tf.Variable 'dense/kernel:0' shape=(10, 1) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32_ref>]

In [9]:
with tf.name_scope('loss'):
    _sub = (y_pred - y)
    _pow = tf.pow(_sub, 2)
    mean_square_error  = tf.reduce_mean(_pow, axis=1)
    mse_loss = tf.reduce_mean(mean_square_error)

In [10]:
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(mse_loss)

In [11]:
tf.summary.scalar('mse', mse_loss)
merged = tf.summary.merge_all()

In [12]:
import random

def iter_data(batch_size):
    batch_x1, batch_x2, batch_y = [], [], []
    for i in range(10000):
        x1 = random.randint(0,9)
        x2 = random.randint(x1, 9)
        y = x1+x2
    
        if len(batch_x1)< batch_size:
            batch_x1.append(x1)
            batch_x2.append(x2)
            batch_y.append([y])
        else:
            yield batch_x1, batch_x2, batch_y
            batch_x1, batch_x2, batch_y = [], [], []

In [13]:
for item in iter_data(2):
    print(item)
    break

([4, 4], [7, 9], [[11], [13]])


In [14]:
from tensorflow.contrib.tensorboard.plugins import projector

def save_metadata(file):
    with open(file, 'w') as f:
        f.write('obj\tname\n')
        for i in range(10):
            f.write('{}\t{}\n'.format(i, i))


In [ ]:
log_dir = './trboard'
with tf.Session() as sess:
    saver = tf.train.Saver()
    summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
    
    tf.global_variables_initializer().run()
    for i,(batch_x1, batch_x2, batch_y) in enumerate(iter_data(batch_size=10)):
        _, mse, _summary = sess.run(
            [train_op, mse_loss, merged],
            feed_dict={
                x1: batch_x1,
                x2: batch_x2,
                y: batch_y,
            }
        )
        summary_writer.add_summary(_summary, i)
        
    config = projector.ProjectorConfig()
    embed= config.embeddings.add()
    embed.tensor_name = 'number_embedding/embedding:0'
    embed.metadata_path = 'metadata.tsv'
    projector.visualize_embeddings(summary_writer, config)
    
    save_metadata(os.path.join(log_dir, embed.metadata_path))
    saver.save(sess, os.path.join(log_dir, 'a_model.ckpt'))


In [ ]:
!tensorboard --logdir=./trboard

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/talipov/MyProjects/projectX/acosta_env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 0.4.0rc3 at http://MBP-Ruslan-2.dvm.int.ridero.eu:6006 (Press CTRL+C to quit)
W0228 16:30:19.831342 Reloader plugin_event_multiplexer.py:203] Deleting accumulator '.'
